# 01 — Through-the-Cycle (TTC) PD Model

**Author:** Devesh Gupta  
**Purpose:** Build a defendable TTC PD modelling notebook following the portfolio flow: Data Strategy → EDA → Feature Engineering → Modelling → Calibration → Validation → Governance → Monitoring.

> **Note / assumption:** I do **not** have access to your internal systems or production datasets. This notebook uses *placeholder file paths* under `data/`. Replace those CSVs with your actual internal exports (application data, repayment history, bureau scores, and macro series). Any time I make an assumption (file names, column names, default flags), I mark it clearly so you can change it to your real field names.

## Quick instructions before running
1. Upload your actual CSVs to the repository `data/` folder or change the `DATA_*` path variables below.
2. Required inputs (example names used below — **replace** if different):
   - `data/applications.csv` — borrower/application-level static fields (customer_id, origination_date, annual_income, credit_limit, etc.)
   - `data/repayments.csv` — transaction-level / performance table (customer_id, txn_date, dpd, etc.) or precomputed flag `dpd_90_within_12m`
   - `data/bureau.csv` — external bureau scores (customer_id, bureau_score, bureau_date) — optional but highly recommended
   - `data/macro.csv` — monthly macro indicators (date, gdp_growth, unemployment, cpi, etc.)
3. Open the notebook in Colab: `File -> Save a copy in Drive` then run cells top → bottom. Replace field names where noted.

In [ ]:
# Imports & global config
import os
import sys
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid', context='notebook')
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, brier_score_loss
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve
import statsmodels.api as sm
RND = 42
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 140)
print('Python', sys.version)


## 1. Data Strategy & Governance (Load + basic checks)

*Purpose:* establish defensible inputs and basic DQ checks. Replace the example filenames with your real exports.

In [ ]:
# === CONFIG: update these to match your repo/data filenames ===
DATA_APPLICATIONS = 'data/applications.csv'          # static application/profile data
DATA_REPAYMENTS = 'data/repayments.csv'            # transaction or performance table OR precomputed flags
DATA_BUREAU = 'data/bureau.csv'                    # optional: bureau scores
DATA_MACRO = 'data/macro.csv'                      # monthly macro indicators (date, gdp_growth, unemployment...)
# =================================================================

def must_read(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"File not found: {path} — please upload to repo data/ or change path in the config cell.")
    return pd.read_csv(path)

# Try loading with defensive messages
for p in [DATA_APPLICATIONS, DATA_REPAYMENTS, DATA_MACRO]:
    if not os.path.exists(p):
        print(f"WARNING: {p} not found. Replace with your actual file or upload sample. This notebook will still show code examples.")

# Load when available
apps = pd.read_csv(DATA_APPLICATIONS, parse_dates=['origination_date']) if os.path.exists(DATA_APPLICATIONS) else pd.DataFrame()
repay = pd.read_csv(DATA_REPAYMENTS, parse_dates=['txn_date']) if os.path.exists(DATA_REPAYMENTS) else pd.DataFrame()
macro = pd.read_csv(DATA_MACRO, parse_dates=['date']) if os.path.exists(DATA_MACRO) else pd.DataFrame()
bureau = pd.read_csv(DATA_BUREAU, parse_dates=['bureau_date']) if os.path.exists(DATA_BUREAU) else pd.DataFrame()

print('applications rows:', len(apps), 'repayments rows:', len(repay), 'macro rows:', len(macro), 'bureau rows:', len(bureau))


### 1.1 Scoping, default definition & time windows (edit as required)

Assumptions in this notebook (edit below if your policies differ):
- Portfolio: retail unsecured (single facility per `customer_id` assumed)
- Default definition: `90+ DPD within 12 months of origination` mapped to `dpd_90_within_12m` flag OR computed from `repay` table.
- Performance horizon: 12 months from origination (you can change to 24/36 months by editing code where `12` appears)


In [ ]:
### Create a modeling dataframe `df` that merges required inputs and builds the label
if apps.empty:
    print('No applications file loaded — creating a small synthetic demo sample so rest of notebook runs. Replace with your real data.')
    # Minimal demo data (only for demonstration). Replace with your real exports.
    apps = pd.DataFrame({
        'customer_id':[1,2,3,4,5,6,7,8,9,10],
        'origination_date':pd.to_datetime(['2018-01-01','2018-02-10','2018-03-01','2019-06-05','2019-06-10','2020-01-05','2020-02-20','2021-03-03','2021-05-05','2021-07-01']),
        'annual_income':[40000,30000,120000,55000,45000,38000,60000,48000,130000,75000],
        'credit_limit':[20000,15000,50000,25000,20000,15000,60000,30000,80000,40000],
        'delinq_2yrs':[0,1,0,2,0,0,3,0,0,1]
    })
if repay.empty:
    # Synthetic flag column: dpd_90_within_12m
    # In your real data, compute this by looking at repayment DPDs within the 12 month window from origination.
    apps['dpd_90_within_12m'] = [0,1,0,0,1,0,0,0,1,0]
    df = apps.copy()
else:
    # Example: if you have repayment rows with dpd or status, you'd compute the 90+ DPD within 12 months here.
    # This code assumes a simple pre-computed flag exists in repay or apps. If not, replace with your label-building logic.
    if 'dpd_90_within_12m' in apps.columns:
        df = apps.copy()
    else:
        # fallback: try to aggregate repay to compute the label (example pattern)
        print('Computing dpd_90_within_12m from repayments table — adjust logic for your schema')
        repay['orig_month'] = repay['txn_date'].dt.to_period('M').astype(str)
        # Example aggregation: max dpd by customer in 12 months after orig
        # NOTE: Replace with your real orig date join logic
        df = apps.merge(repay.groupby('customer_id').agg(max_dpd=('dpd','max')).reset_index(), on='customer_id', how='left')
        df['dpd_90_within_12m'] = (df['max_dpd'] >= 90).astype(int)

df['orig_month'] = pd.to_datetime(df['origination_date']).dt.to_period('M').astype(str)
print('Modeling df sample:')
display(df.head())


## 2. Exploratory Data Analysis (EDA)
Understand cohort volumes, observed default rates, and relationships with candidate predictors.

In [ ]:
# Cohort size and observed default rate
if 'dpd_90_within_12m' not in df.columns:
    raise KeyError('Label dpd_90_within_12m not found in df — compute it from repayments or add the flag to apps file.')
cohort = (df.groupby('orig_month')
            .agg(n=('customer_id','size'),
                 default_rate=('dpd_90_within_12m','mean'))
            .reset_index().sort_values('orig_month'))
print('Cohort summary (first 20 rows):')
display(cohort.head(20))

plt.figure(figsize=(12,4))
plt.plot(cohort['orig_month'], cohort['default_rate'], marker='o')
plt.xticks(rotation=45)
plt.title('Observed Default Rate by Origination Month')
plt.ylabel('Default rate')
plt.xlabel('Origination month')
plt.show()


In [ ]:
# Univariate checks for a few candidate variables
# Example variables: annual_income, delinq_2yrs, credit_limit
def binned_default_rate(df, col, bins=5):
    try:
        df['_b'] = pd.qcut(df[col].rank(method='first'), q=bins, duplicates='drop')
    except Exception:
        df['_b'] = pd.cut(df[col], bins=bins)
    out = df.groupby('_b').agg(n=('customer_id','size'), default_rate=('dpd_90_within_12m','mean'))
    return out

for c in ['annual_income','credit_limit','delinq_2yrs']:
    if c in df.columns:
        print('\nVariable:', c)
        display(binned_default_rate(df, c, bins=5))


## 3. Data Transformation & Feature Engineering
Derive stable predictors and perform cyclical adjustment (example: remove relationship with GDP from a behavioral variable).

In [ ]:
# Create basic transformed features (edit names to your real columns)
df['log_income'] = np.log1p(df['annual_income']) if 'annual_income' in df.columns else 0
df['util_rate'] = (df['credit_limit'] - 0) / df['credit_limit'] if 'credit_limit' in df.columns else 0  # placeholder; replace with revolving_balance/credit_limit
df['past_delinquency_flag'] = (df['delinq_2yrs'] > 0).astype(int) if 'delinq_2yrs' in df.columns else 0

# Merge macro at orig_month for cyclical adjustment — requires macro file with 'date' column at monthly frequency
if not macro.empty:
    macro['orig_month'] = macro['date'].dt.to_period('M').astype(str)
    # choose one macro variable for demo (gdp_growth). Replace with your macro names.
    if 'gdp_growth' not in macro.columns:
        # if macro doesn't have gdp_growth, try any numeric column
        numeric_macros = macro.select_dtypes(include=[np.number]).columns.tolist()
        if numeric_macros:
            chosen_macro = numeric_macros[0]
            print('Using macro column', chosen_macro, 'for cyclical adjustment (no gdp_growth column found).')
            macro = macro.rename(columns={chosen_macro: 'gdp_growth'})
        else:
            print('Macro file loaded but contains no numeric columns. Skipping cyclical adjustment demo.')
    df = df.merge(macro[['orig_month','gdp_growth']].drop_duplicates(), on='orig_month', how='left')
else:
    df['gdp_growth'] = 0

# Demonstration: cyclical adjustment — regress util_rate on gdp_growth and retain residuals as cycle-removed feature
if 'util_rate' in df.columns:
    Xc = sm.add_constant(df['gdp_growth'].fillna(0))
    yc = df['util_rate'].fillna(0)
    cyc_mod = sm.OLS(yc, Xc).fit()
    df['util_rate_resid'] = cyc_mod.resid
    print('Cyclical regression summary (util_rate ~ gdp_growth):')
    print(cyc_mod.summary().tables[1])
else:
    df['util_rate_resid'] = 0

display(df[[col for col in df.columns if col.startswith(('log_income','util_rate','past_delinquency','gdp_growth','util_rate_resid'))]].head())


## 4. Model Design & Estimation
Logistic regression for interpretability. For TTC we exclude contemporaneous macro dummies and use cycle-removed predictors (residuals) and/or set macro variables to long-run means during calibration.

In [ ]:
# Prepare X and y; choose features that are stable (cycle-adjusted where applicable)
features = []
if 'log_income' in df.columns:
    features.append('log_income')
if 'util_rate_resid' in df.columns:
    features.append('util_rate_resid')
if 'past_delinquency_flag' in df.columns:
    features.append('past_delinquency_flag')

if not features:
    raise ValueError('No modelling features found in df — please create features or edit the feature list.')

X = df[features].fillna(0)
y = df['dpd_90_within_12m'].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RND, stratify=y)
print('Train shape:', X_train.shape, 'Test shape:', X_test.shape)

model = LogisticRegression(class_weight='balanced', solver='liblinear', random_state=RND)
model.fit(X_train, y_train)
y_pred_proba = model.predict_proba(X_test)[:,1]
auc = roc_auc_score(y_test, y_pred_proba)
brier = brier_score_loss(y_test, y_pred_proba)
print(f'AUC: {auc:.4f}    Brier: {brier:.4f}')

coef_df = pd.DataFrame({'feature':features, 'coef': model.coef_.ravel()})
display(coef_df)


## 5. Calibration & Scaling (map model mean to long-run PD)
We often want TTC PDs aligned to a long-run observed average default rate. Here is a simple scaling example — more sophisticated approaches (Platt scaling, isotonic regression, grade-level scaling) are recommended for production.

In [ ]:
# Example: compute long-run observed average default (user should compute using >=1 full cycle of history)
observed_long_run_pd = None
if 'dpd_90_within_12m' in df.columns:
    observed_long_run_pd = df['dpd_90_within_12m'].mean()
    print('Observed long-run PD (naive):', observed_long_run_pd)
else:
    observed_long_run_pd = 0.03  # default assumption — change to your long-run value
    print('Using assumed long-run PD =', observed_long_run_pd)

# Current model average predicted PD on the test set
raw_avg = y_pred_proba.mean()
print('Model raw average PD on test set:', raw_avg)

if raw_avg > 0:
    scale_factor = observed_long_run_pd / raw_avg
else:
    scale_factor = 1.0
print('Scale factor to align to long-run PD:', round(scale_factor,4))

df_test = X_test.copy()
df_test['pd_raw'] = model.predict_proba(X_test)[:,1]
df_test['pd_scaled'] = df_test['pd_raw'] * scale_factor
display(df_test[['pd_raw','pd_scaled']].head())


## 6. Model Validation
Discrimination (AUC), calibration plot, and placeholders for PSI / backtests.

In [ ]:
# Calibration curve
prob_true, prob_pred = calibration_curve(y_test, y_pred_proba, n_bins=10)
plt.figure(figsize=(6,5))
plt.plot(prob_pred, prob_true, marker='o')
plt.plot([0,1],[0,1],'k--')
plt.xlabel('Mean predicted probability')
plt.ylabel('Observed fraction of positives')
plt.title('Calibration plot')
plt.show()

# AUC printed earlier; KS and Gini can be added. Example: compute simple KS
from sklearn.metrics import roc_curve
fpr, tpr, thr = roc_curve(y_test, y_pred_proba)
ks = max(abs(tpr - fpr))
print('KS:', round(ks,4), 'AUC:', round(auc,4))


In [ ]:
# Population Stability Index (PSI) helper — compare distribution in train vs. current / baseline
def psi(expected, actual, buckets=10):
    expected = np.asarray(expected)
    actual = np.asarray(actual)
    breakpoints = np.percentile(expected, np.linspace(0,100,buckets+1))
    eps = 1e-6
    def wgt_pct(arr, low, high):
        return ((arr >= low) & (arr < high)).sum() / len(arr)
    psi_val = 0
    for i in range(len(breakpoints)-1):
        e = wgt_pct(expected, breakpoints[i], breakpoints[i+1]) + eps
        a = wgt_pct(actual, breakpoints[i], breakpoints[i+1]) + eps
        psi_val += (e - a) * np.log(e / a)
    return psi_val

psi_val = psi(X_train['log_income'] if 'log_income' in X_train else X_train.iloc[:,0],
              X_test['log_income'] if 'log_income' in X_test else X_test.iloc[:,0], buckets=10)
print('Example PSI on log_income (train vs test):', round(psi_val,4))

# Backtesting / vintage checks placeholder — implement based on your business vintage definitions
print('Backtesting: compare predicted vs observed defaults by origination vintage — implement using your production vintage logic.')


## 7. Documentation & Governance (Checklist)
- Model Development Document (MDD): record data sources, feature engineering, model specification, assumptions, and limitations.  
- Validation Report: independent tests and backtests; record KS, AUC, calibration, PSI, coefficient stability.  
- Version control: commit code + data snapshot (where permitted).  
- Sign-offs: model owner, validator, governance committee.  
- Data lineage: store copies or reproducible extraction scripts with timestamps.

## 8. Implementation & Monitoring (Operational notes)
- Score export: save `pd_scaled` per customer to `artifacts/score_YYYYMMDD.csv` for ingestion (example code below).  
- Monitoring: create monthly dashboard tracking AUC, KS, PSI, calibration drift, and changes to long-run PD.  
- Recalibration triggers: material performance deterioration (AUC drop > X), PSI > 0.2 on key variables, or a regime change in macro indicators.


In [ ]:
# Example: export scaled PDs for the test slice (production integration step)
out = df_test.copy()
out = out.reset_index().rename(columns={'index':'_orig_index'})
out['customer_id'] = df.loc[out['_orig_index'], 'customer_id'].values if '_orig_index' in out.columns else None
OUT_PATH = 'artifacts/pd_scores_sample.csv'
os.makedirs('artifacts', exist_ok=True)
out[['pd_raw','pd_scaled']].to_csv(OUT_PATH, index=False)
print('Wrote sample PD scores to', OUT_PATH)


### Summary & Next steps
- This notebook is an end-to-end TTC PD *template*. Replace placeholder filenames and column names with your real data exports.  
- Next notebook (PiT PD) will follow the same structure but keep macro variables to current values and include time-series features and survival modelling options.  
- If you want, I can convert this JSON into the actual `.ipynb` file content and paste it into your repo file content for you to commit (or provide the raw JSON as a downloadable file).